In [30]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import yfinance as yf
from datetime import datetime
from sklearn.covariance import LedoitWolf


In [31]:
TICKERS = [
    # Large-cap tech & communication
    "AAPL","MSFT","GOOGL","AMZN","TSLA","META","NVDA","NFLX","ADBE","CRM",

    # Semiconductors
    "INTC","AMD","QCOM","AVGO","TXN","MU","AMAT","LRCX","NXPI","KLAC",

    # Banks & financials
    "JPM","BAC","WFC","C","GS","MS","V","MA","PYPL","AXP",

    # Healthcare & pharma
    "PFE","JNJ","MRK","UNH","ABBV","LLY","BMY","GILD","AMGN","REGN",

    # Energy
    "XOM","CVX","BP","SHEL","TTE","COP","SLB","HAL","MPC","PSX",

    # Consumer staples & discretionary
    "PG","KO","PEP","WMT","COST","HD","MCD","NKE","TGT","SBUX",

    # Industrials
    "BA","CAT","DE","HON","GE","UPS","MMM","LMT","RTX","FDX",

    # Utilities
    "NEE","DUK","SO","AEP","EXC","SRE","WEC","PEG","ED","XEL",

    # Telecom
    "VZ","T","TMUS","CMCSA","CHTR","DIS","FOX","FOXA","ROKU",

    # More large-caps for diversity
    "ORCL","IBM","SAP","CSCO","MMM","BKNG","SPGI","BLK","MO","PM",
    "ADSK","WDAY"
]

In [32]:

END = datetime.today().strftime("%Y-%m-%d")
RISK_FREE_RATE = 0.035
ANNUAL_TRADING_DAYS = 252
OUTPUT_DIR = "optiverse_outputs"

L2_REG = 1e-2
EPS_JITTER = 1e-5
ALLOW_SHORT = False
N_POINTS = 50
os.makedirs(OUTPUT_DIR, exist_ok=True)

LOOKBACK_YEARS = [3, 5, 10]
MIN_DATA_FRACTION = 0.9

In [33]:
#this function is used to download the prices from yfinance for the following stocks in tickers from the dates containing from start to end.
#and placing them properly in a dataframe.
def download_prices(tickers, start, end=END):
    raw = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=False)
    if raw.empty:
        raise RuntimeError("yfinance returned no data for the given tickers / date range.")
    if isinstance(raw.columns, pd.MultiIndex):
        if "Adj Close" in raw.columns.get_level_values(0):
            df = raw["Adj Close"]
        elif "Close" in raw.columns.get_level_values(0):
            df = raw["Close"]
        else:
            df = raw.xs(raw.columns.levels[0][-1], axis=1, level=0, drop_level=True)
    else:
        if "Adj Close" in raw.columns:
            df = raw["Adj Close"]
        elif "Close" in raw.columns:
            df = raw["Close"]
        else:
            df = raw
    if isinstance(df, pd.Series):
        df = df.to_frame()
    df = df.dropna(how="all")
    if list(df.columns) != list(tickers) and len(df.columns) == len(tickers):
        try:
            df.columns = tickers
        except Exception:
            pass
    return df

In [34]:
#this function is compute_returns, we know using markowitz , the returns are computed from mean, and risk from covaraince
# and we use Ledoit-wolf shrinkage to make the covarience matrix to make it close to a target matrix, like the identity matrix, to create a more stable and well-conditioned estimate
def compute_returns(prices, use_shrinkage=True):
    rets = prices.pct_change().dropna()
    mean_daily = rets.mean()
    cov_daily = rets.cov()
    mu = mean_daily * ANNUAL_TRADING_DAYS
    Sigma = cov_daily * ANNUAL_TRADING_DAYS

    if use_shrinkage:
        try:
            lw = LedoitWolf().fit(rets.values)
            Sigma = pd.DataFrame(lw.covariance_, index=prices.columns, columns=prices.columns)
            Sigma = Sigma * ANNUAL_TRADING_DAYS
        except Exception as e:
            print("Ledoit-Wolf shrinkage failed, using sample covariance. Error:", e)

    # Adding jitter to diagonal to avoid singularity
    Sigma = Sigma + EPS_JITTER * np.eye(Sigma.shape[0])

    return rets, mu.values.reshape(-1, 1), Sigma


In [35]:

def solve_markowitz(mu, Sigma, R_target, allow_short=False, l2_reg=0.0):
    n = Sigma.shape[0]
    w = cp.Variable((n, 1))
    objective = cp.quad_form(w, Sigma)
    if l2_reg and l2_reg > 0:
        objective = objective + l2_reg * cp.sum_squares(w)
    constraints = [cp.sum(w) == 1, mu.T @ w >= R_target]
    if not allow_short:
        constraints.append(w >= 0)
    prob = cp.Problem(cp.Minimize(objective), constraints)

    # Trying OSQP with stricter tolerances first
    try:
        prob.solve(solver=cp.OSQP, verbose=False, max_iter=200000, eps_abs=1e-6, eps_rel=1e-6)
    except Exception:
        pass
    # fallback to ECOS
    if prob.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        try:
            prob.solve(solver=cp.ECOS, verbose=False, max_iters=100000)
        except Exception:
            pass
    # last fallback to SCS
    if prob.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        try:
            prob.solve(solver=cp.SCS, verbose=False, max_iters=25000)
        except Exception:
            pass

    if prob.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        w_opt = np.array(w.value).flatten()
    else:
        w_opt = np.full(n, np.nan)
    return w_opt, prob.status